## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
keras = tf.keras

## Meus dados WIN

In [ ]:
path = r'C:\Users\adria\1Python\Meu_novo_projeto\WIN_N_1K\WIN_N_1k.csv'
raw = pd.read_csv(path, index_col=0, parse_dates=True)

In [ ]:
df = pd.DataFrame()
df['close'] = raw['close']

In [ ]:
plt.plot(df['close'].iloc[:200])

In [ ]:
#df['r'] = df['close'].pct_change()

In [ ]:
df = df.dropna()

In [ ]:
series = df['close'].values

In [ ]:
time = np.arange(len(series))

In [ ]:
split_time = 80000
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

## Moving Average

In [ ]:
def moving_average_forecast(series, window_size):
  """Forecasts the mean of the last few values.
     If window_size=1, then this is equivalent to naive forecast
     This implementation is *much* faster than the previous one"""
  mov = np.cumsum(series)
  mov[window_size:] = mov[window_size:] - mov[:-window_size]
  return mov[window_size - 1:-1] / window_size

In [ ]:
janela = 2

In [ ]:
moving_avg = moving_average_forecast(series, janela)[split_time - janela:]

In [ ]:
len(moving_avg)

In [ ]:
moving_avg

In [ ]:
len(x_valid)

In [ ]:
x_valid

In [ ]:
plt.plot(x_valid[:10], linestyle="-", color="blue")
plt.plot(moving_avg[:10], linestyle="--", color="red")
plt.grid(True)
plt.show()

In [ ]:
diff_series = (series[4:] - series[:-4])
diff_time = time[4:]

In [ ]:
diff_moving_avg = moving_average_forecast(diff_series, 2)[split_time - 4 - 4:]

In [ ]:
len(diff_moving_avg)

In [ ]:
diff_moving_avg

In [ ]:
diff_moving_avg_plus_past = series[split_time-2 - 4:-4] + diff_moving_avg

In [ ]:
len(diff_moving_avg_plus_past)

In [ ]:
diff_moving_avg_plus_past

In [ ]:
tempo = 50

plt.plot(x_valid[:tempo], linestyle="-", color="blue")
plt.plot(moving_avg[:tempo], linestyle="--", color="red")
plt.plot(diff_moving_avg_plus_past[:tempo], linestyle="-.", color="green")
plt.grid(True)
plt.show()

In [ ]:
x_valid.shape

In [ ]:
diff_moving_avg_plus_past[2:].shape

In [ ]:
keras.metrics.mean_absolute_error(x_valid, diff_moving_avg_plus_past[2:]).numpy()

In [ ]:
keras.metrics.mean_squared_error(x_valid, diff_moving_avg_plus_past[2:]).numpy()

## DesRetorne os dados e verifique

In [ ]:
df2 = pd.DataFrame()

In [ ]:
df2['diff_moving_avg_plus_past'] = diff_moving_avg_plus_past

In [ ]:
df2['x_valid'] = x_valid

In [ ]:
df2

In [ ]:
df['close'].iloc[79999]

### desfazendo o x_valid

In [ ]:
ultimo_preco_fechamento = 113630.0

In [ ]:
df2['preco_valid'] = (1 + df2['x_valid']).cumprod()

In [ ]:
df2['preco_valid'] = ultimo_preco_fechamento * df2['preco_valid']

In [ ]:
df2

### desr a previsão

In [ ]:
df2['desr_prev'] = (1 + df2['diff_moving_avg_plus_past']).cumprod()

In [ ]:
df2['desr_prev'] = ultimo_preco_fechamento * df2['desr_prev']

In [ ]:
df2

In [ ]:
df2['diff'] = df2['preco_valid']-df2['desr_prev']

In [ ]:
df2

In [ ]:
df2['diff'].mean()

In [ ]:
plt.plot(df2['preco_valid'].iloc[:100], linestyle="-", color="b")
plt.plot(df2['desr_prev'].iloc[:100], linestyle="-", color="red")

plt.show()